# Project : Part I 
- Steps mentioned below
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.

Use the Notebook to build the code to scrape the following Wikipedia page,

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe like the 

one shown below:



     

In [7]:
# !pip install geopy
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim


In [3]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
# soup
table = soup.find('tbody')
rows= table.select('tr')


In [1]:
row = []
for r in rows:
    row.append(r.get_text())

In [6]:
df= pd.DataFrame(row)

### table create and cleaning stepwise


In [7]:
df1 = df[0].str.split('\n',expand =True)

In [8]:
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()
df3.shape

(180, 7)

### remove data having not assigned borough

In [9]:
df4 = df3[df3.Borough != 'Not assigned']
df4.shape

(103, 7)

### remove data duplicates

In [2]:
df5 = df4.groupby(['Postal Code', "Borough"], sort=  False ).agg(','.join)
df5.reset_index(inplace = True)

In [11]:
df5.shape

(103, 7)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       181 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB


# Part - II 

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking  postal code M5G as an example, your code would look something like this:

In [13]:
url ="http://cocl.us/Geospatial_data"
df6 = pd.read_csv(url)
df6.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df6  = pd.merge(df5,df6, on='Postal Code')

# Part 3 :- Solution 

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
to generate maps to visualize your neighborhoods and how they cluster together. 
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [3]:
tor = df6[df6['Borough'].str.contains("Toronto")]   # only Toronto contains in borough

In [29]:

address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [48]:
Tor_map = folium.Map(location=[latitude, longitude], zoom_start=8)

In [49]:
for lat, lng, borough, neighborhood in zip(tor['Latitude'], tor['Longitude'], 
                                           tor['Borough'], tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='black',
        fill_opacity=0.8,
        parse_html=False).add_to(Tor_map)  
Tor_map  